In [ ]:
!pip install transformers==4.40.1 accelerate==0.30.0 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 5.2 MB/s eta 0:00:00
   ━━━

# 8.4절 실습: LLM 서빙 프레임워크

## 예제 8.1. 실습에 사용할 데이터셋 준비

In [ ]:
import torch
from datasets import load_dataset

# dataset로부터 프롬프트셋 만드는 함수
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

In [ ]:
dataset

,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
...,...,...,...,...,...
107,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,캐릭터가 완료한 퀘스트 중 보상 경험치가 100 이상 200 이하인 퀘스트의 이름과...,"SELECT q.name, q.reward_items FROM quests AS q...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
108,1,CREATE TABLE characters (\n character_id INT ...,"경험이 1000에서 2000 사이인 캐릭터들의 이름, 레벨, 경험치, 아이템 이름 ...","SELECT c.name, c.level, c.experience, i.item_n...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
109,1,CREATE TABLE npcs (\n npc_id INT PRIMARY KEY ...,각 역할별로 npc의 총 수를 알려주세요.,"SELECT role, COUNT(npc_id) FROM npcs GROUP BY ...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
110,1,CREATE TABLE characters (\n character_id INT ...,장비가 장착된 상태인 캐릭터의 플레이어 ID와 아이템 이름을 나열하십시오.,"SELECT T1.player_id, T2.item_name FROM charact...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


## 예제 8.2. 모델과 토크나이저를 불러와 추론 파이프라인 준비

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
# 양자화 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# 문장 생성으로 수행할 작업을 선택, 모델과 토크나이저를 전달하여 파이프라인 생성
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 예제 8.3. 배치 크기에 따른 추론 시간 확인

배치 크기가 증가하면서 추론 시간이 감소

In [ ]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

1: 291.64935398101807
2: 378.6555185317993
4: 259.971853017807
8: 185.87119841575623
16: 138.6426601409912
32: 123.08963108062744


## 예제 8.4. vLLM 모델 불러오기

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [ ]:
import torch
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)

INFO 09-02 10:40:14 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='shangrilar/yi-ko-6b-text2sql', speculative_config=None, tokenizer='shangrilar/yi-ko-6b-text2sql', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-02 10:40:14 utils.py:608] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 09-02 10:40:16 selector.py:65] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 09-02 10:40:16 selector.py:33] Using XFormers backend.
INFO 09-02 10:40:17 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 09-02 10:41:19 model_runner.py:173] Loading model weights took 11.5127 GB
INFO 09-02 10:41:21 gpu_executor.py:119] # GPU blocks: 75, # CPU blocks: 4096
INFO 09-02 10:41:24 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-02 10:41:24 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to d

## 예제 8.5. vLLM을 활용한 오프라인 추론 시간 측정

기존 LLM에 비해 더 감소된 시간 <br/>
vLLM에서 max_num_seqs가 배치 크기라 보면 됨

In [ ]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs
  # temperature, top_p가 클수록 더 랜덤한 답변이 생성
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
  print(f'{max_num_seqs}: {time.time() - start_time}')

Processed prompts: 100%|██████████| 112/112 [05:06<00:00,  2.73s/it]


1: 306.3442621231079


Processed prompts: 100%|██████████| 112/112 [02:10<00:00,  1.16s/it]


2: 130.44945526123047


Processed prompts: 100%|██████████| 112/112 [01:29<00:00,  1.25it/s]


...

8: 87.33036828041077


Processed prompts: 100%|██████████| 112/112 [01:25<00:00,  1.30it/s]


16: 86.01092076301575


Processed prompts: 100%|██████████| 112/112 [01:31<00:00,  1.22it/s]

32: 91.52784562110901


## 예제 8.6. 온라인 서빙을 위한 vLLM API 서버 실행

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 1024

INFO 09-03 04:04:31 api_server.py:151] vLLM API server version 0.4.1
INFO 09-03 04:04:31 api_server.py:152] args: Namespace(host='127.0.0.1', port=8888, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, served_model_name=None, lora_modules=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], model='shangrilar/yi-ko-6b-text2sql', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=1024, guided_decoding_backend='outlines', worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=1, max_parallel_loading_workers=None, ray_workers_use_nsight=False, block_size=16, enable_prefix_ca

## 예제 8.7. 백그라운드에서 vLLM API 서버 실행하기

예제 8.6를 강제 종료하고 실행 <br/>
예제 8.6을 실행하지 않고 실행하면 다운로드하는 시간이 추가로 소요됨

In [ ]:
!nohup python -m vllm.entrypoints.openai.api_server \
# --model 뒤에 model id를 전달
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 512 &

nohup: appending output to 'nohup.out'


## 예제 8.8. API 서버 실행 확인

예제 8.7에서 서버가 준비되려면 시간이 걸려 제대로된 output이 나올 때까지 기다렸다가 실행

In [ ]:
# curl은 서버에 요청을 보냄, /v1/models 은 vLLM 프레임워크에서 API 서버에서 사용할 수 있는 모델 정보를 제공
!curl http://localhost:8888/v1/models

{"object":"list","data":[{"id":"shangrilar/yi-ko-6b-text2sql","object":"model","created":1725336937,"owned_by":"vllm","root":"shangrilar/yi-ko-6b-text2sql","parent":null,"permission":[{"id":"modelperm-ca3351af6744469e9b8db249ef348625","object":"model_permission","created":1725336937,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

## 예제 8.9. API 요청

In [ ]:
import json

# LLM 생성에 사용될 모델 이름, 입력 프롬프트 및 설정 정보 담기
json_data = json.dumps(
    {"model": "shangrilar/yi-ko-6b-text2sql",
      "prompt": dataset.loc[0, "prompt"],
      "max_tokens": 128,
      "temperature": 1}
    )

# 요청에 대한 출력을 반환
!curl http://localhost:8888/v1/completions \
    -H "Content-Type: application/json" \
    -d '{json_data}'

{"id":"cmpl-49bdcdc903c94c9e812783781192b01d","object":"text_completion","created":1725336966,"model":"shangrilar/yi-ko-6b-text2sql","choices":[{"index":0,"text":"SELECT reward_items, SUM(reward_experience) FROM quests GROUP BY reward_items;","logprobs":null,"finish_reason":"stop","stop_reason":null}],"usage":{"prompt_tokens":127,"total_tokens":149,"completion_tokens":22}}

## 예제 8.10. OpenAI 클라이언트를 사용한 API 요청

In [ ]:
from openai import OpenAI

# vLLM 서버의 주소와 API 키
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8888/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="shangrilar/yi-ko-6b-text2sql",
                                 prompt=dataset.loc[0, 'prompt'], max_tokens=128)
print("생성 결과:", completion.choices[0].text)

생성 결과: SELECT reward_items, SUM(reward_experience) AS reward_experience_total FROM quests GROUP BY reward_items;
